In [43]:
import pandas as pd
import MeCab
import json
from os import listdir
from os.path import isfile, join
import math
import neologdn
import unicodedata
import re

In [56]:
japaneseMasculineWordsList = open('./data/gendered_words_lists/ja/masculine_raw.json', 'r')
japaneseMasculineWords = json.load(japaneseMasculineWordsList)
japaneseMasculineNounsList = open('./data/gendered_words_lists/ja/masculine_directive.json', 'r')
japaneseMasculineNouns = json.load(japaneseMasculineNounsList)
japaneseMasculineNouns = []
japaneseFeminineWordsList = open('./data/gendered_words_lists/ja/feminine_raw.json', 'r')
japaneseFeminineWords = json.load(japaneseFeminineWordsList)
japaneseFeminineNounsList = open('./data/gendered_words_lists/ja/feminine_directive.json', 'r')
japaneseFeminineNouns = json.load(japaneseFeminineNounsList)
japaneseFeminineNouns = []


In [54]:
# Create DataFrame Object From Job Advertisements Data To Calculate The Gender Bias Score
dataDirectory = './data/job_advertisements/ja/'
columnNames = ['title', 'summary', 'detail', 'desired_person', 'occupation', 'mascline_score', 'feminine_score']
df = pd.DataFrame(index=[], columns=columnNames)
files = [f for f in listdir(dataDirectory) if isfile(join(dataDirectory, f)) and ('failed' not in f and 'error' not in f)]

tagger = MeCab.Tagger('-d /opt/homebrew/lib/mecab/dic/mecab-ipadic-neologd')
tagger.parse('')
re_hiragana = re.compile(r'^[あ-ん]+$')
masculineWords = []
feminineWords = []

for file in files:
    jsonOpen = open(dataDirectory + file, 'r')
    jobAdvertisements = json.load(jsonOpen)
    fileName = file.replace('.json', '')
    dataIndex = 1
    for jobAdvertisement in jobAdvertisements:
        if jobAdvertisement.get(str(dataIndex)):
            # clean the sentences
            cleaned_title = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(dataIndex)]['title'].replace(' |\n', ''))).lower()
            cleaned_summary = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(dataIndex)]['description_summary'].replace(' |\n', ''))).lower()
            cleaned_detail = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(dataIndex)]['description_detail'].replace(' |\n', ''))).lower()
            cleaned_desired_person = neologdn.normalize(unicodedata.normalize("NFKC", jobAdvertisement[str(dataIndex)]['desired_person'].replace(' |\n', ''))).lower()            
            all_sentences =  cleaned_title + ' ' + cleaned_summary + ' ' + cleaned_detail + ' ' + cleaned_desired_person 
            
            node = tagger.parseToNode(all_sentences)
            masculineScore = 0
            feminineScore = 0
            counter = 0
            masculineWordAndOccupation = []
            feminineWordAndOccupation = []
            while node:
                counter = counter + 1
                #指定した品詞(category)のみ原型で抽出
                if node.feature.split(",")[0] in  ['名詞', '形容詞', '動詞']:
                    if node.feature.split(",")[0] == '': 
                        None
                    elif len(node.feature.split(",")[6]) == 1: 
                        None
                    elif re_hiragana.fullmatch(node.feature.split(",")[6]):
                        None
                    else: 
                        for japaneseMasculineWord in japaneseMasculineWords:
                            if japaneseMasculineWord == node.feature.split(",")[6]:
                                masculineScore += 1
                                masculineWordAndOccupation.append({str(dataIndex):[japaneseMasculineWord, fileName]})
                                break
                        for japaneseMasculineNoun in japaneseMasculineNouns:
                            if japaneseMasculineNoun == node.feature.split(",")[6]:
                                masculineScore += 2
                                masculineWordAndOccupation.append({str(dataIndex):[japaneseMasculineNoun, fileName]})
                                break
                        for japaneseFeminineWord in japaneseFeminineWords:
                            if japaneseFeminineWord == node.feature.split(",")[6]:
                                feminineScore += 1
                                feminineWordAndOccupation.append({str(dataIndex):[japaneseFeminineWord, fileName]})
                                break
                        for japaneseFeminineNoun in japaneseFeminineNouns:
                            if japaneseFeminineNoun == node.feature.split(",")[6]:
                                feminineScore += 2
                                feminineWordAndOccupation.append({str(dataIndex):[japaneseFeminineNoun, fileName]})
                                break
                else:
                    pass
                node = node.next
            if masculineScore == 0 and feminineScore == 0:
                x = 0
            else:
                x = (1/(masculineScore + feminineScore)) * (masculineScore - feminineScore)
            genderBiasScore = 1 / (1 + math.exp(-x))
                        
            record =  pd.DataFrame({
                'title': cleaned_title,
                'summary': cleaned_summary,
                'detail': cleaned_detail,
                'desired_person': cleaned_desired_person,
                'occupation': fileName,
                'mascline_score': masculineScore,
                'feminine_score': feminineScore,
                'gender_bias_score': genderBiasScore
            }, index=[str(dataIndex)])
            masculineWords.append(masculineWordAndOccupation)
            feminineWords.append(feminineWordAndOccupation)

            df = pd.concat([df, record])
        dataIndex += 1


In [51]:
df.groupby('occupation').mean()

,gender_bias_score
occupation,
beautician,0.426398
carpentor,0.454646
childminder,0.435318
driver,0.435217
electrician,0.447987
nurse,0.435948
pharmacist,0.457806
secretary,0.421792
security_guard,0.434178


In [57]:
print(masculineWords)

print(feminineWords)

[[{'1': ['決める', 'childminder']}, {'1': ['評価', 'childminder']}, {'1': ['自主性', 'childminder']}, {'1': ['体育', 'childminder']}, {'1': ['目標', 'childminder']}, {'1': ['見る', 'childminder']}, {'1': ['資格', 'childminder']}, {'1': ['持つ', 'childminder']}, {'1': ['可能', 'childminder']}, {'1': ['チャレンジ', 'childminder']}], [{'2': ['目的', 'childminder']}, {'2': ['チャレンジ', 'childminder']}, {'2': ['個性', 'childminder']}, {'2': ['合わせる', 'childminder']}, {'2': ['指導', 'childminder']}, {'2': ['自由', 'childminder']}, {'2': ['無理', 'childminder']}, {'2': ['無理', 'childminder']}, {'2': ['持つ', 'childminder']}, {'2': ['作る', 'childminder']}, {'2': ['良い', 'childminder']}, {'2': ['資格', 'childminder']}, {'2': ['持つ', 'childminder']}, {'2': ['立つ', 'childminder']}], [{'3': ['資格', 'childminder']}, {'3': ['可能', 'childminder']}, {'3': ['自由', 'childminder']}, {'3': ['体育', 'childminder']}, {'3': ['体育', 'childminder']}, {'3': ['資格', 'childminder']}, {'3': ['持つ', 'childminder']}, {'3': ['資格', 'childminder']}, {'3': ['持つ', 'childminde